In [2]:
import pandas as pd
import numpy as np
import os
import lxml.etree as etree
import re

## Read File

In [3]:
file_path = '1987-2016.kml'
# read file
ty_name=[]
intensity=[]
mslp=[]
max_int=[]
min_slp=[]
timesteps=[]
lon=[]
lat=[]
parser = etree.XMLParser(strip_cdata=False)
tree = etree.parse(file_path, parser=parser)
#print(etree.tostring(tree, xml_declaration=True))
#print(tree.xpath('//kml:Placemark/kml:TimeSpan/text()', namespaces={"kml":"http://www.opengis.net/kml/2.2"}))
nmsp = '{http://www.opengis.net/kml/2.2}'

## Data Cleansing & Data Preparation

In [4]:
### walk through history typhoon file ###
for fd in tree.iterfind('/{0}Document/{0}Folder/{0}Folder/{0}Folder'.format(nmsp)):
    temp = fd.find('{0}Placemark/{0}name'.format(nmsp)).text
    ty_name.append(temp)
    count=0
    ## feature extraction ##
    for dscp in fd.iterfind('{0}Placemark'.format(nmsp)):
        #print(dscp.find('{0}description'.format(nmsp)).text)
        dscript = dscp.find('{0}description'.format(nmsp)).text
        dscript = re.split(r'<[^>]+>|[\s\n\t]', dscript)
        temp_info= np.array([ele for ele in dscript if len(ele)>0])
        #print(len(temp_info))
        if len(temp_info) == 26:
            temp_info=np.append(temp_info,count)
            timesteps=np.append(timesteps,count)
            intensity=np.append(intensity,[temp_info[13]])
            mslp=np.append(mslp,[temp_info[16]])
            max_int=np.append(max_int,[temp_info[20]])
            min_slp=np.append(min_slp,[temp_info[24]])
            count+=1
        elif len(temp_info) == 30:
            timesteps=np.append(timesteps,count)
            intensity=np.append(intensity,[temp_info[17]])
            mslp=np.append(mslp,[temp_info[20]])
            max_int=np.append(max_int,[temp_info[24]])
            min_slp=np.append(min_slp,[temp_info[28]])
            count+=1
        elif len(temp_info) == 33:
            timesteps=np.append(timesteps,count)
            intensity=np.append(intensity,[temp_info[17]])
            mslp=np.append(mslp,[temp_info[23]])
            max_int=np.append(max_int,[temp_info[27]])
            min_slp=np.append(min_slp,[temp_info[31]])
            count+=1
        elif len(temp_info) == 35:
            timesteps=np.append(timesteps,count)
            intensity=np.append(intensity,[temp_info[20]])
            mslp=np.append(mslp,[temp_info[25]])
            max_int=np.append(max_int,[temp_info[29]])
            min_slp=np.append(min_slp,[temp_info[33]])
            count+=1
    for i in range(count,105):
        timesteps=np.append(timesteps,[i], axis=0)
        intensity= np.append(intensity, [0], axis = 0)
        mslp= np.append(mslp, [0], axis = 0)
        max_int= np.append(max_int, [0], axis = 0)
        min_slp= np.append(min_slp, [0], axis = 0)
        count+=1
### Geogaphic coordinate feature extraction ###

for fd in tree.iterfind('/{0}Document/{0}Folder/{0}Folder/{0}Folder'.format(nmsp)):
    count=0
    for coor in fd.iterfind('{0}Placemark/{0}Point'.format(nmsp)):
        geocoor = coor.find('{0}coordinates'.format(nmsp)).text
        lontitude,latitude,_ = geocoor.split(',')
        lon = np.append(lon,lontitude)
        lat = np.append(lat,latitude)
        count+=1
    for i in range(count,105):
        lon = np.append(lon,lon[-1])
        lat = np.append(lat,lat[-1])
    #print(len(lon))
lon = np.asanyarray(lon, dtype=np.float)
lat = np.asanyarray(lat, dtype=np.float)
intensity = np.asanyarray(intensity, dtype=np.float)
mslp = np.asanyarray(mslp, dtype=np.float)
max_int = np.asanyarray(max_int, dtype=np.float)
min_slp = np.asanyarray(min_slp, dtype=np.float)

for i in range(len(lon)):
    if lon[i] < 0:
        lon[i] = lon[i]+360
for i in range(len(mslp)):
    if mslp[i] == 9999:
        mslp[i] = mslp[np.where((mslp!=9999)|(mslp!=0))].mean()

lon_train, lon_test = lon[:71505], lon[-23835:]
lat_train, lat_test = lat[:71505], lat[-23835:]
intensity_train, intensity_test = intensity[:71505], intensity[-23835:]
mslp_train, mslp_test = mslp[:71505], mslp[-23835:]
max_int_train, max_int_test = max_int[:71505], max_int[-23835:]
min_slp_train, min_slp_test = min_slp[0:71505], min_slp[-23835:]

#  Dataframe for training
data_train = {'lon':lon_train,'lat':lat_train,'Intensity':intensity_train}
data_test = {'lon':lon_test,'lat':lat_test,'Intensity':intensity_test}
X_train = pd.DataFrame(data=data_train)
X_test = pd.DataFrame(data=data_test)
# target Dataframe
y_train = X_train.shift(1)
y_train = pd.DataFrame(data=y_train)
y_train.fillna(0, inplace=True)
y_test = X_test.shift(1)
y_test = pd.DataFrame(data=y_test)
y_test.fillna(0, inplace=True)

## Save File

In [6]:
X_train.to_csv('X_train.csv', index=False, encoding='utf-8')
X_test.to_csv('X_test.csv', index=False, encoding='utf-8')
y_train.to_csv('y_train.csv', index=False, encoding='utf-8')
y_test.to_csv('y_test.csv', index=False, encoding='utf-8')

In [10]:
df = pd.read_csv('X_train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71505 entries, 0 to 71504
Data columns (total 3 columns):
lon          71505 non-null float64
lat          71505 non-null float64
Intensity    71505 non-null float64
dtypes: float64(3)
memory usage: 1.6 MB
